In [ ]:
import geopandas as gpd
import pandas as pd
import seaborn as sns
import shapely.geometry as sg
import numpy as np
import pyproj
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import joblib as jl
import traffic
from traffic.core import Traffic


%matplotlib inline

In [ ]:
unc_asp_tfc_gdf = pd.read_pickle('../data/southeng/southeng_unc_asp_tfc_2019.pkl.bz2', compression='bz2')
# unc_asp_tfc_gdf = unc_asp_tfc_gdf.to_crs('epsg:3857')
unc_asp_tfc_gdf[unc_asp_tfc_gdf.select_dtypes(np.float16).columns] = unc_asp_tfc_gdf.select_dtypes(
        np.float16).astype(np.float32)

In [ ]:
unc_asp_tfc_gdf[unc_asp_tfc_gdf['altitude'] < 304 * 3].flight_id.nunique()

In [ ]:
traj_gdf = gpd.read_file('../data/test_traj/QA-IOW.geojson').set_crs('epsg:4326').to_crs('epsg:27700')
traj_alt = 1000 #m
lat_buffer = 1000 #m
vert_buffer = 500 #m
traj_gdf

In [ ]:
unc_asp_tfc_gdf

In [ ]:
traj_poly = traj_gdf.buffer(lat_buffer).iloc[0]
traj_poly_gdf = gpd.GeoDataFrame(geometry=[traj_poly])
traj_poly

In [ ]:
x_res = 1000
y_res = 1000
unc_asp_tfc = Traffic(unc_asp_tfc_gdf)
tfc_unc_xy_gdf = unc_asp_tfc_gdf.to_crs('epsg:27700')
tfc_unc_xy_gdf = tfc_unc_xy_gdf.assign(x=tfc_unc_xy_gdf.geometry.x, y=tfc_unc_xy_gdf.geometry.y)

In [ ]:
tfc_agg = tfc_unc_xy_gdf.assign(
    x=lambda elt: (elt.x // x_res) * x_res,
    y=lambda elt: (elt.y // y_res) * y_res,
).groupby(["x", "y"]).agg(altitude_mean=pd.NamedAgg('altitude', np.nanmean),
                          altitude_std=pd.NamedAgg('altitude', np.std), track_mean=pd.NamedAgg('track', np.nanmean),
                          track_std=pd.NamedAgg('track', np.nanstd),
                          groundspeed_mean=pd.NamedAgg('groundspeed', np.nanmean),
                          groundspeed_std=pd.NamedAgg('groundspeed', np.nanstd),
                          flight_id_nunique=('flight_id', 'nunique'))
tfc_agg['flight_id_nunique'] = tfc_agg['flight_id_nunique']/0.21739

In [ ]:
def make_centrepoint_box(x, y, x_res, y_res):
    half_x_res = x_res/2
    half_y_res = y_res/2
    return sg.box(x-half_x_res, y-half_y_res, x+half_x_res, y+half_y_res)

tfc_agg_gdf = tfc_agg.reset_index()
# tfc_agg_gdf = gpd.GeoDataFrame(tfc_agg_gdf, geometry=gpd.points_from_xy(tfc_agg_gdf.x, tfc_agg_gdf.y, crs='epsg:3857'))
tfc_agg_gdf = gpd.GeoDataFrame(tfc_agg_gdf, geometry=[make_centrepoint_box(x, y, x_res, y_res) for x,y in zip(tfc_agg_gdf.x, tfc_agg_gdf.y)] , crs='epsg:27700')
# sg.box

In [ ]:
print(np.mean(np.diff(tfc_agg_gdf.x) % x_res))
print(np.mean(np.diff(tfc_agg_gdf.y) % y_res))

In [ ]:
env_gdf = tfc_agg_gdf[tfc_agg_gdf.within(traj_poly.envelope)]

In [ ]:
env_tfc_gdf =  tfc_unc_xy_gdf[tfc_unc_xy_gdf.within(traj_poly.envelope)]
# env_tfc_gdf.plot.scatter('x', 'y')

In [ ]:
env_tfc_gdf.flight_id.nunique()

In [ ]:
from traffic.core import Traffic
import tqdm
tfc = Traffic(env_tfc_gdf[env_tfc_gdf['altitude'] < 304*2].drop(labels=['geometry', 'name', 'type', 'icaoClass'], axis=1))
res = 1000
resampled_flights = []
for flight in tfc:
    try:
        print(flight.data.shape)
        # print(pd.infer_freq(pd.DatetimeIndex(flight.data.timestamp)))
        # print(flight.max('groundspeed'))
        print(int(np.floor(res/flight.max('groundspeed'))))
        # resampled_flights.append(flight.resample(f"{2*int(max(np.floor(res/flight.max('groundspeed')), 1))}S"))
        resampled_flights.append(flight.filter().resample(f"10S"))
    except:
        continue

print('resampled')
resampled_tfc = Traffic.from_flights(resampled_flights)
print('serialising')
resampled_tfc.to_pickle('../data/southeng/southeng_qa_iow_resample_10s_tfc_2019.pkl.bz2', compression='bz2')

In [ ]:
from traffic.core import Traffic
import tqdm
tfc = Traffic(env_tfc_gdf[env_tfc_gdf['altitude'] < 304*2].drop(labels=['geometry', 'name', 'type', 'icaoClass'], axis=1))
res = 1000
# resampled_flights = []
def resample(flight):
        # print(flight.data.shape)
        # print(pd.infer_freq(pd.DatetimeIndex(flight.data.timestamp)))
        # print(flight.max('groundspeed'))
        # print(int(np.floor(res/flight.max('groundspeed'))))
        # resampled_flights.append(flight.resample(f"{2*int(max(np.floor(res/flight.max('groundspeed')), 1))}S"))
        return flight.filter().resample(f"30S")

resampled_flights = jl.Parallel(n_jobs=-2, verbose=10)(jl.delayed(resample)(f) for f in tfc)

resampled_tfc = Traffic.from_flights(resampled_flights)
print('serialising')
resampled_tfc.to_pickle('../data/southeng/southeng_qa_iow_resample_10s_tfc_2019.pkl.bz2', compression='bz2')

In [ ]:
lat_intersect_gdf = tfc_agg_gdf[tfc_agg_gdf.intersects(traj_poly)]
# d3_intersect_gdf = lat_intersect_gdf[(lat_intersect_gdf['altitude'] <= traj_alt+vert_buffer) & (lat_intersect_gdf['altitude'] >= traj_alt-vert_buffer)]

In [ ]:
lat_raw_tf_intersect_gdf = unc_asp_tfc_gdf[unc_asp_tfc_gdf.intersects(traj_poly)]

In [ ]:
lat_raw_tf_intersect_gdf['hour'] = pd.DatetimeIndex(lat_raw_tf_intersect_gdf['timestamp']).hour
lat_raw_tf_intersect_gdf['month'] = pd.DatetimeIndex(lat_raw_tf_intersect_gdf['timestamp']).month

In [ ]:
lat_raw_tf_intersect_gdf

In [ ]:
from cartes.crs import LambertConformal, EPSG_27700, PlateCarree, EuroPP, Mercator, Projection
from traffic.drawing import countries, lakes, ocean
from traffic.data import airports
bounds = (-2.9, 1.5, 50.5, 51.9)

fig, ax = plt.subplots(
    1, 1, figsize=(12,4), subplot_kw=dict(projection=Projection('epsg:3857')),
)

ax.add_feature(countries())
ax.add_feature(lakes())
ax.add_feature(ocean())
# ax.set_extent(bounds)
# ax.set_global()
tfc_magg = tfc_agg

xs = np.sort(tfc_magg['flight_id_nunique'].reset_index()['x'].unique().astype(int))
ys = np.sort(tfc_magg['flight_id_nunique'].reset_index()['y'].unique().astype(int))

pcm = ax.pcolormesh(xs, ys, np.log(tfc_magg['flight_id_nunique'].reset_index().pivot_table('flight_id_nunique', 'y', 'x', fill_value=np.nan))
, cmap='inferno', alpha=0.5)

# ax.add_geometries([traj_poly], 'epsg:3857', facecolor='red', edgecolor='red', alpha=0.3)

airports['EGHL'].point.plot(ax, alpha=0.2)
airports['EGTK'].point.plot(ax, alpha=0.2)
airports['EGKA'].point.plot(ax, alpha=0.2)
airports['EGMD'].point.plot(ax, alpha=0.2)
airports['EGTB'].point.plot(ax, alpha=0.2)
airports['EGHO'].point.plot(ax, alpha=0.2)
airports['EGBP'].point.plot(ax, alpha=0.2)
airports['EGKH'].point.plot(ax, alpha=0.2)

cb = fig.colorbar(pcm)
cb.set_label('ln Traffic Counts')
ax.set_title('UK Uncontrolled Airspace Traffic Density 2019, <5000ft')

fig.tight_layout()
fig.savefig('full_unc_counts.svg')

In [ ]:
traj_poly.bounds

In [ ]:
from cartes.crs import LambertConformal, EPSG_27700, PlateCarree, EuroPP, Mercator, Projection
from traffic.drawing import countries, lakes, ocean
import shapely.geometry as sg
bounds = (-2.9, 1.5, 50.5, 51.9)

fig, ax = plt.subplots(
    1, 1, figsize=(12,6), subplot_kw=dict(projection=Projection('epsg:3857')),
)

ax.add_feature(countries())
ax.add_feature(lakes())
ax.add_feature(ocean())
ax.set_extent((-1.32, -0.91, 50.69, 50.86))#[traj_poly.bounds[0],traj_poly.bounds[2], traj_poly.bounds[1],traj_poly.bounds[3]])
# ax.set_global()

xs = np.sort(tfc_agg_gdf[tfc_agg_gdf.within(traj_poly.envelope)]['x'].unique().astype(int))
ys = np.sort(tfc_agg_gdf[tfc_agg_gdf.within(traj_poly.envelope)]['y'].unique().astype(int))

ax.add_geometries([traj_poly], 'epsg:3857', facecolor='red', edgecolor='red', alpha=0.3)

pcm = ax.pcolormesh(xs, ys, np.log(tfc_agg_gdf[tfc_agg_gdf.within(traj_poly.envelope)].pivot_table('flight_id_nunique', 'y', 'x', fill_value=np.nan))
, cmap='inferno', alpha=0.5)


cb = fig.colorbar(pcm)
cb.set_label('ln Traffic Counts')
ax.set_title('UK Uncontrolled Airspace Traffic Density 2019, <5000ft')

fig.tight_layout()
fig.savefig('geom_unc_counts.png')

In [ ]:
cell_vol = x_res * y_res * 304.8 * 5 # Up to 5000ft
FT2M = 0.3048
HRS_IN_YR = 8766
buffer_mean_tfc_dens = lat_intersect_gdf['flight_id_nunique'].sum()/(traj_poly.area*2*vert_buffer)/HRS_IN_YR
buffer_max_tfc_dens = lat_intersect_gdf['flight_id_nunique'].max()/cell_vol/HRS_IN_YR
print(f'Along trajectory buffer:')
print(f'Mean traffic density is {buffer_mean_tfc_dens:.6} aircraft/m^3/hour equiv to 1 aircraft per {np.sqrt(1/(buffer_mean_tfc_dens*500*FT2M))/9:.6}m x 500ft in an hour')
print(f'Max traffic density is {buffer_max_tfc_dens:.6} aircraft/m^3/hour equiv to 1 aircraft per {np.sqrt(1/(buffer_max_tfc_dens*500*FT2M))/9:.6}m x 500ft in an hour')


In [ ]:
n_lis_sq = len(lat_raw_tf_intersect_gdf[lat_raw_tf_intersect_gdf['squawk'].isin(['4306', '7011'])])
print(f'{n_lis_sq} aircraft are on local listening squawks. This is {n_lis_sq/len(lat_raw_tf_intersect_gdf):.1%} of aircraft along trajectory')

### Traffic Motion Distributions:

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(12,6),
)

(lat_raw_tf_intersect_gdf['altitude']/FT2M).hist(ax=ax, density=True, bins=8)
ax.set(title='Traffic altitude along trajectory buffer', xlabel='Altitude [ft] 500ft Bins', ylabel='Freq Density')

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(12,6),
)

(lat_raw_tf_intersect_gdf['track']).hist(ax=ax, density=True, bins=18)
ax.set(title='Traffic track along trajectory buffer', xlabel='Track [deg] 20deg Bins', ylabel='Freq Density')

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(12,6),
)

(lat_raw_tf_intersect_gdf['groundspeed']).hist(ax=ax, density=True)
ax.set(title='Traffic speed along trajectory buffer', xlabel='Speed [m/s]', ylabel='Freq Density')

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(12,6),
)

(lat_raw_tf_intersect_gdf['hour']).hist(ax=ax, density=True, bins=24)
ax.set(title='Traffic encounter Hour of Day', xlabel='Hour of Day', ylabel='Freq Density')

## Probability of Collision:
## The Ideal Gas method
For this section, we assume the traffic becomes an ideal gas, such that we can model it with the usual ideal gas law:

$$pV=nRt=n k_B N_A T = N k_B T$$

Obviously, we aren't dealing with an actual gas, therefore Boltzmann's relations between energy and temperature are not useful for us here. We can rewrite this using our own variables:

$$ p_{MAC} = \rho_t A_C |\vec{\overline{v_R}}| t$$

where:
- $p_{MAC}$ is the probability of mid air collision for within a period $t$
- $\rho_t$ is the air traffic density
- $A_C$ is the projected collision area. This is assumed to be the maximum cross section of an aircraft.
- $\vec{\overline{v_R}}$ is the mean relative velocity vector between aircraft.
- $t$ is the time horizon

Next, we make some assumptions

In [ ]:
v_ac = 80 # ~250kts
A_C = 20*15
t = 1 # density is already per hour

In [ ]:
v_rel_max = lat_raw_tf_intersect_gdf['groundspeed'].max() + v_ac # assume worst case head on
v_rel_mean = lat_raw_tf_intersect_gdf['groundspeed'].astype(float).mean() + v_ac # assume worst case head on
p_mac_max = float(buffer_max_tfc_dens) * A_C * v_rel_max * t
p_mac_mean = float(buffer_mean_tfc_dens) * A_C * v_rel_mean * t
print(f'Unmitigated Max Collision Probability is {p_mac_max:.3e} per flight hour, equiv to a single collision every {1/(p_mac_max*HRS_IN_YR):.3} years.')
print(f'Unmitigated Mean Collision Probability is {p_mac_mean:.3e} per flight hour, equiv to a single collision every {1/(p_mac_mean*HRS_IN_YR):.4} years.')

## Agent Based Simulation Method

Here we use a simulated environment that is formed of 2 main components:

- Background Traffic
- Ownship Agent

As we are not really looking at the collision rate of the existing aircraft already flying around, instead we are interested in the potential collisions between our own aircraft/UAS (ownship) and the existing traffic. Therefore the simulation is divided along this line.

The background traffic is generated based on the traffic motion distributions for the area of operation (ie the ones above) and the maximum traffic density (although this can be mean as well, however we are angling for a feasible worst case).

The ownship then follows the trajectory path being analysed and any conflicts are recorded. This simulation is performed a great number of times following a Monte Carlo methodology. The conflict occurences can then be aggregated to form overall statistics for the expected collision rate specific to that operation.

In [ ]:
from abs_specific import Simulation, Traffic, OwnshipAgent
import scipy.stats as ss
import os

# import dask
# from dask.distributed import Client, LocalCluster
#
# cluster = LocalCluster()
# client = Client()

# import ray
# ray.init(log_to_driver=False, ignore_reinit_error=True)

In [ ]:
ownship_velocity = 70 #m/s
target_sim_hrs = 2.7e4
sim_per_batch = 500

# Sim compute setup
n_cores = int(os.cpu_count())
target_sim_secs = target_sim_hrs * 60 * 60
path_length = traj_gdf.iloc[0][0].length
exp_secs_per_sim = path_length / ownship_velocity
exp_n_sims = np.ceil(target_sim_secs / exp_secs_per_sim)
exp_n_jobs = int(np.ceil(exp_n_sims/sim_per_batch))
print(f'{exp_n_jobs} jobs at {sim_per_batch} simulations per job')

In [ ]:
path_length

In [ ]:
tb = traj_poly.envelope.bounds
sim_bounds = [tb[0], tb[2], tb[1], tb[3], 0, 1524]
sim_tfc_density = 1e-9 #tfc_agg_gdf[tfc_agg_gdf.within(traj_poly.envelope)]['flight_id_nunique'].max()/cell_vol/HRS_IN_YR
env_tfc = unc_asp_tfc_gdf[unc_asp_tfc_gdf.intersects(traj_poly.envelope)]
env_vel_kde = ss.gaussian_kde(env_tfc['groundspeed'])
env_track_kde = ss.gaussian_kde(env_tfc['track'])

own_path = np.hstack((np.array(traj_gdf.iloc[0][0].coords), traj_alt * np.ones((len(traj_gdf.iloc[0][0].coords)))[:, None]))

own_args = (own_path, ownship_velocity)
tfc_args = (sim_bounds, sim_tfc_density, env_vel_kde, ss.norm(0, 2), env_track_kde)
steps = 5000

In [ ]:
def run_sim(own_args, tfc_args, steps):
    seed = int.from_bytes(os.urandom(8), byteorder="big") % ((2**32) - 1)
    bg_tfc = Traffic(*tfc_args, seed=seed)
    own = OwnshipAgent(*own_args)

    spec_sim = Simulation(bg_tfc, own, steps=steps, seed=seed, conflict_dists=(20,20))
    spec_sim.run()
    # print(hash(spec_sim.traffic.positions.data.tobytes()))
    return spec_sim


def run_batch_sim(batch_size, own_args, tfc_args, steps):
    sims = [run_sim(own_args, tfc_args, steps) for _ in range(batch_size)]
    sim_ids = [s.sim_id for s in sims]
    sim_seeds = [s.sim_seed for s in sims]
    sims_conflict_sums = [s.conflict_log for s in sims]
    sims_times = [s.end_timestep for s in sims]
    db_data = zip(sim_ids, sim_seeds, sims_times, sims_conflict_sums)

    dbcon = sqlite3.connect('../data/abs-specific.db')
    dbcur = dbcon.cursor()
    dbcur.executemany("INSERT INTO sims VALUES (?,?,?,?)", db_data)
    dbcon.commit()

    # return sims

# sims = [run_sim(own_args, tfc_args, steps) for _ in range(20)]

# jl.Parallel(n_jobs=n_cores, verbose=15, )(jl.delayed(run_batch_sim)(sim_per_batch, own_args, tfc_args, steps) for _ in range(exp_n_jobs))

# sims = [dask.delayed(run_batch_sim)(sim_per_batch, own_args, tfc_args, steps) for _ in range(exp_n_jobs)]
# dask.compute(*sims)

# sims = [run_batch_sim.remote(sim_per_batch, own_args, tfc_args, steps) for _ in range(exp_n_jobs)]
# ray.get(sims)

In [ ]:
# Setup the database for logging of runs
import sqlite3
dbcon = sqlite3.connect('../data/abs-specific.5e-11.db')
dbcur = dbcon.cursor()
dbcur.execute("CREATE TABLE IF NOT EXISTS sims(id, seed, timesteps, n_conflicts)")

In [ ]:
res = dbcur.execute("SELECT * FROM sims").fetchall()

In [ ]:
sim_data = np.array(res)[:, 2:]
sim_data = np.cumsum(sim_data, axis=0)
n_sims = sim_data.shape[0]
n_conflicts = sim_data[-1,1]
sim_data = np.hstack((sim_data, (sim_data[:, 1] / (sim_data[:, 0]/3600))[:, None]))

In [ ]:
print(f'Probability: {n_conflicts/n_sims:2e}')

In [ ]:
data = np.array(res)[:, 2:]
means = data.mean(axis=0)
means[0] /= 3600
print(f'{means[1]/means[0]:2e}')

In [ ]:
print(sim_data.shape[0], ' sims run')
print(f'{sim_data.shape[0] * exp_secs_per_sim/3600:.2e} hours run' )

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(12,6),
)

ax.plot(range(sim_data.shape[0]), sim_data[:, 2])
ax.axhline(np.median(sim_data[:,2]), color='red', linestyle=':')
# ax.set_xscale('symlog')
ax.set_yscale('log')
ax.set_title("ABS convergence for Traffic Density 5e-11 m$^{-3}$ hr$^{-1}$")
ax.set_xlabel("Operations Simulated")
ax.set_ylabel("uMAC Risk/hour")
print(f'Median uMAC rate {np.median(sim_data[:,2]):2e} per hour')

In [ ]:
conflict_sum = sim_data[:, 2].sum()
sim_hrs = sim_data[:, 1].sum() / (60*60)
sim_ops = sim_data.shape[0]
mean_col_rate = conflict_sum / sim_hrs


print(f'{sim_hrs} flight hrs of operations simulated equiv. to {exp_n_sims} individual operations')
print(f'Max unmitigated collision rate {conflict_sum / sim_hrs:.2e} collisions per hour')
print(f'Max unmitigated collision rate {conflict_sum / sim_ops:.2e} collisions per operation or one unmitigated collision per {sim_ops / conflict_sum} operations')